# Beginning of music analysis project

# 1 Survey of available datasets

**interesting links**<br>
1) https://www.juanmontesinos.com/Solos/<br>
2) https://www.upf.edu/web/mtg/irmas<br> 
3) https://www.kaggle.com/data/65258<br> 

notes on Dataset \#1 (solos)
- appears to be links to a bunch of youtube samples

notes on Dataset #2 (IRMAS-Sample)<br>
- training and test sets already separated<br>
- each datapoint is a .wav file and text file with label<br>


# 1 Following librosa quickstart

## VERY VERY IMPORTANT NOTE: LIBROSA DOES NOT WORK WITH THE LATEST VERSION OF PYTHON (3.9)- I am using 3.8 for this project

In [ ]:
import librosa
import matplotlib.pyplot as plt

In [ ]:
# Get filepath to an audio example
filename = librosa.example('nutcracker')

In [ ]:
from IPython.display import Audio

In [ ]:
# load audio as a waveform (y), load the sample rate as sr
# by default: mono, sample rate = 22050Hz
y, sr = librosa.load(filename)

In [ ]:
plt.plot(y)

In [ ]:
test = Audio(y, rate=sr)

In [ ]:
test

In [ ]:
# Run the default beat tracker
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

In [ ]:
tempo

In [ ]:
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

In [ ]:
beat_times

## More advanced usage

In [ ]:
# load example clip
y, sr = librosa.load(librosa.ex('nutcracker'))

# set hop length
hop_length = 512

# separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

In [ ]:
plt.plot(y_harmonic)
plt.title("harmonic")
plt.show()

plt.plot(y_percussive)
plt.title("percussive")
plt.show()

In [ ]:
Audio(y_harmonic, rate=sr)

In [ ]:
Audio(y_percussive, rate=sr)

In [ ]:
# beat track on percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

In [ ]:
tempo

In [ ]:
# compute MFCC features from raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

In [ ]:
mfcc

**Info on MFCC:** https://en.wikipedia.org/wiki/Mel-frequency_cepstrum

In [ ]:
# Add first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)

In [ ]:
mfcc_delta

In [ ]:
import numpy as np

In [ ]:
# stack and synchronize between beat events
# use the mean value instead of median
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), 
                                   beat_frames)

# Compute chroma feature from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

In [ ]:
# Aggregate chroma feature between beat events
# use the median value of each feature between beat frames
beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)

In [ ]:
# stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

## 1.2 Messing around with other tracks

converter: https://ontiva.com/en/youtube-to-wav-converter

Source: https://www.youtube.com/watch?v=C9vN6nZawvU

In [ ]:
y2, sr = librosa.load("Ensiferum_Twilight_Tavern.wav")

In [ ]:
Audio(y2, rate=sr)

In [ ]:
plt.plot(y2)

In [ ]:
y2_harmonic, y2_percussive = librosa.effects.hpss(y2)
Audio(y2_harmonic, rate=sr)

In [ ]:
Audio(y2_percussive, rate=sr)

harmonic: lead guitars, strings, bass, half of the rhythm guitar, and part of the vocals<br>
percussive: kick / snare drums, harsh vocals<br>

Source: https://www.youtube.com/watch?v=-C5LcGNLppk

In [ ]:
y_aal, sr = librosa.load("AAL_Tempting_Time.wav", sr=44100)
Audio(y_aal, rate=sr)

In [ ]:
y_aal_harmonic, y_aal_percussive = librosa.effects.hpss(y_aal)
Audio(y_aal_harmonic, rate=sr)

In [ ]:
Audio(y_aal_percussive, rate=sr)

Idea: use these to generate tempo maps for absurdly complicated songs (e.g. [The Dance of Eternity](https://www.youtube.com/watch?v=FwkcRTNMsWs))

# 2 Loading some sample data
# 2.1 IRMAS

In [ ]:
ytest, sr = librosa.load("IRMAS-Sample/Testing/14.  Boots Randolph - Yakety Sax-1.wav")
Audio(ytest, rate=sr)

In [ ]:
ytest, sr = librosa.load("IRMAS-Sample/Testing/12 What'll I Do - Bud Shank And Bob-4.wav")
Audio(ytest, rate=sr)

In [ ]:
ytest, sr = librosa.load("IRMAS-Sample/Training/sax/118__[sax][nod][jaz_blu]1702__3.wav")
Audio(ytest, rate=sr)

# 2.2 Solos: https://arxiv.org/abs/2006.07931

Dataset imported using `pip install solos`<br>
https://www.juanmontesinos.com/Solos/python/

In [ ]:
import Solos

In [ ]:
Solos.get_solos_ids()

Copy some code from [youtubesaver.py](https://github.com/JuanFMontesinos/Solos/blob/master/youtubesaver.py) to download one of these videos

In [ ]:
import youtube_dl as _youtube_dl

In [ ]:
video_id = '5w35MH2IP28'

outtmpl = '%(id)s.%(ext)s'
ydl_opts = {
            'format': 'bestvideo+bestaudio',
            'outtmpl': outtmpl,
            """
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            """
            'logger': None
        }

with _youtube_dl.YoutubeDL(ydl_opts) as ydl:
    test = ydl.download(['https://www.youtube.com/watch?v=%s' % video_id])
    print("this is printing so no errors are thrown if ydl.download is commented")

In [ ]:
test

**TODO**: This code downloaded a youtube video into the current directory (mp4). Figure out how to get the audio from this. 

**TODO**: search for other datasets that are audio only

https://brianmcfee.net/papers/ismir2018_openmic.pdf

**TODO**: Figure out what a bunch of these terms in the librosa library actually mean

**Idea**: Look into the source code in librosa for the harmonic / percussion split- this might give use extra ideas for our own work

## update 2/7- running a few things from echorne_draft

In [1]:
from os import walk

In [2]:
acousticGuitars = []
musicFolder = '../../music-data/'
testFolder = 'nsynth-valid/audio/'
for root, dirs, files in walk(musicFolder+testFolder):
    for file in files:
        if file.startswith('guitar_acoustic') and file.endswith('.wav'):
            acousticGuitars.append(file)
    break
acousticGuitars[:20]

[]

# Misc useful links & other tricks

Create environment: `conda create --name <env name>`<br>
Activate environment: `conda activate <env name>`<br>
Install jupyter with `conda install jupyter`, NOT `pip install jupyter`<br>

jupyter notebook not working<br>
https://stackoverflow.com/questions/50687077/cant-run-any-ipynb-files-using-jupyter-lab-or-jupyter-notebook-filenotfounderr<br>

Include button to hide code (I copied Max Masnick's solution in the first part of several of my other projects). Very useful if you're making a write-up for a non-technical audience that doesn't care about the code.<br>
https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer<br>